# LHW2 : Maximum Entropy Markov Models
In this homework, you will be using a first-order maximum entropy Markov model for part-of-speech tagging. You will be implementing Viterbi decoding, featurizing the text for the MEMM, and performing ablation tests.

## Setup
We'll begin by importing the needed models and providing some skeleton code.

In [1]:
import sys 
import math
import numpy as np
from collections import Counter
from sklearn import linear_model
from scipy import sparse
import pickle
import re

# Dictionary to store indices for each feature
feature_vocab = {}

# Dictionary to store the indices of each POS tag
label_vocab = {}


# load up any external resources here
def initialize():
    """ 
        :return (dictionary data - any format that you wish, which
        can be reused in get_features below)
    """
    data = {}
    return data


def load_data(filename):
    """
        load data from filename and return a list of lists
        all_toks = [toks1, toks2, ...] where toks1 is
                a sequence of words (sentence)

        all_labs = [labs1, labs2, ...] where labs1 is a sequence of
                labels for the corresponding sentence
    """
    file = open(filename)
    all_toks = []
    all_labs = []
    toks = []
    labs = []
    for line in file:
        # Skip the license
        if "This data is licensed from" in line:
            continue
        cols = line.rstrip().split("\t")
        if len(cols) < 2:
            all_toks.append(toks)
            all_labs.append(labs)
            toks = []
            labs = []
            continue
        toks.append(cols[0])
        labs.append(cols[1])

    if len(toks) > 0:
        all_toks.append(toks)
        all_labs.append(labs)

    return all_toks, all_labs


def print_message(m):
    num_stars = 10
    if verbose:
        print("*" * num_stars + m + "*" * num_stars)


def decode(Y_pred):
    """
        select the decoding algorithm
    """
    if use_greedy:
        return greedy_decode(Y_pred)
    else:
        return viterbi_decode(Y_pred)

In the next cell there are some constants that will be used for the MEMM. Feel free to change the values of any of the constants or flags if needed.

In [2]:
# Control variables, use for debugging and testing
verbose = True
use_greedy = False

# Minimum number of observations for a feature to be included in model
MINIMUM_FEAT_COUNT = 2 
# L2 regularization strength; range is (0,infinity), with stronger regularization -> 0 (in this package)
L2_REGULARIZATION_STRENGTH = 1e0 

TRAINING_FILE = 'wsj.pos.train'
DEVELOPMENT_FILE = 'wsj.pos.dev'
TEST_FILE = 'wsj.pos.test'
OUTPUT_CSV = 'predictions.csv'

## 1. Viterbi Decoding
Below we have the function for implementing greedy decoding, which we'd like to improve upon by using Viterbi decoding. In the cell following, implement `viterbi_decode`.

In [3]:
def greedy_decode(Y_pred):
    """ 
        greedy decoding to get the sequence of label predictions
    """
    cur = label_vocab["START"]
    preds = []
    for i in range(len(Y_pred)):
        pred = np.argmax(Y_pred[i, cur])
        preds.append(pred)
        cur = pred
    return preds

In [4]:
def viterbi_decode(Y_pred):
    """
        :return
        list of POS tag indices, defined in label_vocab,
        in order of sequence

        :param
        Y_pred: Tensor of shape N * M * L, where
        N is the number of words in the current sequence,
        L is the number of POS tag labels
        M = L + 1, where M is the number of possible previous labels
        which includes L + "START"

        M_{x,y,z} is the probability of a tag (label_vocab[current_tag] = z)
        given its current position x in the sequence and
        its previous tag (label_vocab[previous_tag] = y)

        Consider the sentence - "I have a dog". Here, N = 4.
        Assume that there are only 3 possible tags: {PRP, VBD, NN}
        M_{0, 3, 2} would give you the probability of "I" being a "NN" if
        it is preceded by "START". "START" is the last index of all lablels,
        and in our example denoted by 3.
    """
    (N, M, L) = Y_pred.shape

    # list of POS tag indices to be returned
    path = []

    """
        Type your code below

        Hints:
        step 1. construct a Viterbi matrix to store the
        intermediate joint probabilities
        step 2. construct a matrix to store the backpointers
        to retrieve the path
        step 3. decode and return the path from the Viterbi matrix
        Note: Probabilities get smaller as you multiply them, and may
        vanish. Remember how we have treated chained probabilities
        this entire class.

    """
    viterbi = np.zeros((L,N))
    backpointer = np.zeros((L,N))
    
    for s in range(L):
        viterbi[s,0] = math.log(Y_pred[0][-1][s])
        backpointer[s,0] = 0
    for t in range(1,N):
        for s in range(L):
            prob = [viterbi[prev_tag,t-1]+math.log(Y_pred[t][prev_tag][s]) for prev_tag in range(L)]
            viterbi[s,t] = np.max(prob)
            backpointer[s,t] = np.argmax(prob)

    bestpathprob = np.max(viterbi[:,N-1])
    bestpathpointer = np.argmax(viterbi[:,N-1])
    
    tag = bestpathpointer
    path.append(tag)
    for t in range(N-1,0,-1):
        tag = int(backpointer[tag,t])
        path.append(tag)
    
    return path[::-1]

## 2. Features
In the `get_features` function below, add features for the maximum entropy Markov model to use, much like in short homework 1. A baseline set of features utilizing unigrams, the tag of the previous word, and the end of the sequence is already implemented for you.

In [5]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/waynewu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def get_features(index, sequence, tag_index_1, data):
    """
        :params
        index: the index of the current word in the sequence to featurize

        sequence: the sequence of words for the entire sentence

        tag_index_1: gives you the POS
        tag for the previous word.

        data: the data you have built in initialize()
        to enrich your feature representation. Use data as you see fit.

        :return (feature dictionary)
        features are in the form of {feature_name: feature_value}
        Calculate the values of each feature for a given
        word in a sequence.

        The current implementation returns the following as features:
        the current word, the tag of the previous word, and whether an
        index the the last in the sequence.

    """
    features = {}
    word = sequence[index].lower()
    
    features["UNIGRAM_%s" % word] = 1
    features["PREVIOUS_TAG_%s" % tag_index_1] = 1
    if index == len(sequence) - 1:
        features["LAST_WORD_IN_SEQUENCE"] = 1

    """
        Create your own features here
    
    """
    #ti = current_tag and wi−2,wi-1
    #ti = current_tag and wi+1,wi+2
    for i in range(1,3):
        if index>i:
            features["PREVIOUS_"+str(i)+"_UNIGRAM_%s" % sequence[index-i].lower()] = 1
        if index+i < len(sequence):
            features["NEXT_"+str(i)+"_UNIGRAM_%s" % sequence[index+i].lower()] = 1
    
    #ti contains at least 1-4 character at front
    if re.search(r'^[a-z]{1}', word) != None:
        features["PREFIX_%s" % word[:1]] = 1
    if re.search(r'^[a-z]{2}', word) != None:
        features["PREFIX_%s" % word[:2]] = 1
    if re.search(r'^[a-z]{3}', word) != None:
        features["PREFIX_%s" % word[:3]] = 1
    if re.search(r'^[a-z]{4}', word) != None:
        features["PREFIX_%s" % word[:4]] = 1
    
    #ti contains at least 1-4 character at end
    if re.search(r'[a-z]{1}$', word) != None:
        features["PREFIX_%s" % word[:-1]] = 1
    if re.search(r'[a-z]{2}$', word) != None:
        features["PREFIX_%s" % word[:-2]] = 1
    if re.search(r'[a-z]{3}$', word) != None:
        features["PREFIX_%s" % word[:-3]] = 1
    if re.search(r'[a-z]{4}$', word) != None:
        features["PREFIX_%s" % word[:-4]] = 1
    
    #ti contains more than one upper-case
    if re.search(r'[A-Z]+', sequence[index]) != None:
        features["CONTAIN_ONE_UPPERCASE"] = 1
    #ti contains number or hyphen
    if re.search(r'[0-9\-]+', word) != None:
        features["CONTAIN_NUMBER_OR_HYPHEN"] = 1
    #ti contains '$%{}&()-`'
    if re.search(r'[$%{}&()-]+', word) != None:
        features["CONTAIN_NUMBER_OR_HYPHEN"] = 1
        
    #ti word shape XX-XXX, XX.XX, ''s', ',', end with s, end with ed, end with ing
    if re.search(r'(\w+)-(\w+)', word) != None:
        features["WORD_SHAPE_XX-XXX"] = 1
    if re.search(r'(\d*).(\d*)', word) != None:
        features["WORD_SHAPE_XX.XX"] = 1
    if re.search(r'\'s', word) != None:
        features["WORD_SHAPE_'s"] = 1
    if re.search(r',', word) != None:
        features["WORD_SHAPE_,"] = 1
    if re.search(r's$', word) != None:
        features["END_WITH_S"] = 1
    if re.search(r'ed$', word) != None:
        features["END_WITH_ED"] = 1
    if re.search(r'ing$', word) != None:
        features["END_WITH_ING"] = 1
    
    #ti in STOPWORDS
    if word in STOPWORDS:
        features["STOPWORDS"] =1
    

    return features

With both Viterbi decoding implemented and features added, we can now build and train our MEMM! Run the cell below to define all the necessary functions to complete our model.

In [7]:
def train(filename, data):
    """
        train a model to generate Y_pred
    """
    all_toks, all_labs = load_data(filename)
    vocab = {}

    # X_verbose is a list of feature objects for the entire train dataset.
    # Each feature object is a dictionary defined by
    # get_features and corresponds to a word
    # Y_verbose is a list of labels for all words in the entire train dataset
    X_verbose = []
    Y_verbose = []

    feature_counts=Counter()

    for i in range(len(all_toks[:10000])):
        toks = all_toks[i]
        labs = all_labs[i]
        for j in range(len(toks)):
            prev_lab = labs[j - 1] if j > 0 else "START"
            feats = get_features(j, toks, prev_lab, data)
            X_verbose.append(feats)
            Y_verbose.append(labs[j])

            for feat in feats:
                feature_counts[feat]+=1

    # construct label_vocab (dictionary) and feature_vocab (dictionary)
    # label_vocab[pos_tag_label] = index_for_the_pos_tag
    # feature_vocab[feature_name] = index_for_the_feature
    feature_id = 1
    label_id = 0

    # create unique integer ids for each label and each feature above the minimum count threshold
    for i in range(len(X_verbose)):
        feats = X_verbose[i]
        true_label = Y_verbose[i]

        for feat in feats:
            if feature_counts[feat] >= MINIMUM_FEAT_COUNT:
                if feat not in feature_vocab:
                    feature_vocab[feat] = feature_id
                    feature_id += 1
        if true_label not in label_vocab:
            label_vocab[true_label] = label_id
            label_id += 1

    # START has last id
    label_vocab["START"] = label_id

    # create train input and output to train the logistic regression model
    # create sparse input matrix

    # X is documents x features empty sparse matrix
    X = sparse.lil_matrix((len(X_verbose), feature_id))
    Y = []

    print_message("Number of features: %s" % len(feature_vocab))
    for i in range(len(X_verbose)):
        feats = X_verbose[i]
        true_label = Y_verbose[i]
        for feat in feats:
            if feat in feature_vocab:
                X[i, feature_vocab[feat]] = feats[feat]
        Y.append(label_vocab[true_label])

    # fit model
    log_reg = linear_model.LogisticRegression(C=L2_REGULARIZATION_STRENGTH, penalty='l2')
    log_reg.fit(sparse.coo_matrix(X), Y)

    return log_reg


def test_dev(filename, log_reg, data):
    """
        predict labels using the trained model
        and evaluate the performance of model
    """
    all_toks, all_labs = load_data(filename)

    # possible output labels = all except START
    L = len(label_vocab) - 1

    correct = 0.
    total = 0.

    num_features = len(feature_vocab) + 1

    # for each sequence (sentence) in the test dataset
    for i in range(len(all_toks[:3000])):

        toks = all_toks[i]
        labs = all_labs[i]

        if len(toks) == 0:
            continue

        N = len(toks)

        X_test = []
        # N x prev_tag x cur_tag
        Y_pred = np.zeros((N, L + 1, L))

        # vector of true labels
        Y_test = []

        # for each token (word) in the sentence
        for j in range(len(toks)):

            true_label = labs[j]
            Y_test.append(true_label)

            # for each preceding tag of the word
            for possible_previous_tag in label_vocab:
                X = sparse.lil_matrix((1, num_features))

                feats = get_features(j, toks, possible_previous_tag, data)
                valid_feats = {}
                for feat in feats:
                    if feat in feature_vocab:
                        X[0, feature_vocab[feat]] = feats[feat]

                # update Y_pred with the probabilities of all current tags
                # given the current word, previous tag and other data/feature
                prob = log_reg.predict_proba(X)
                Y_pred[j, label_vocab[possible_previous_tag]] = prob

        # decode to get the predictions
        predictions = decode(Y_pred)

        # evaluate the performance of the model by checking predictions
        # against true labels
        for k in range(len(predictions)):
            if Y_test[k] in label_vocab and predictions[k] == label_vocab[Y_test[k]]:
                correct += 1
            total += 1

        print("Development Accuracy: %.3f (%s/%s)." % (correct / total, correct, total), end="\r")
    

## 3. Ablation Tests
Run the the model with all of your features included in the cell below, and then once each time you remove a feature. Record your results in a table in a pdf.

In [8]:
feature_vocab = {}
print_message("Initialize Data")
data = initialize()
print_message("Train Model")
log_reg = train(TRAINING_FILE, data)

**********Initialize Data**********
**********Train Model**********
**********Number of features: 59743**********


In [9]:
print_message("Test Model")
test_dev(DEVELOPMENT_FILE, log_reg, data)

**********Test Model**********


## 4. Kaggle Submission
After you've implemented Viterbi decoding and add features of your own, first run the block below to define necessary functions, and the run the block after to generate tags on the test dataset. Submit the created csv onto kaggle at the link https://www.kaggle.com/t/a650043b2d564412b388c3bd352e25dd.

In [10]:
def load_test(filename):
    """
        load data from filename and return a list of lists
        all_toks = [toks1, toks2, ...] where toks1 is
                a sequence of words (sentence)
    """
    file = open(filename)
    all_toks = []
    toks = []
    for line in file:
        # Skip the license
        if "This data is licensed from" in line:
            continue
        col = line.rstrip().split("\t")[0]
        if col == '':
            all_toks.append(toks)
            toks = []
            continue
        toks.append(col)

    if len(toks) > 0:
        all_toks.append(toks)

    return all_toks


def write_predictions(datafile, outputfile, log_reg, data):
    """
        predict labels using the trained model
        and evaluate the performance of model
    """
    all_toks = load_test(datafile)
    index_to_label = {v: k for k, v in label_vocab.items()}

    # possible output labels = all except START
    L = len(label_vocab) - 1
    num_features = len(feature_vocab) + 1
    n = 1

    with open(outputfile, 'w') as f:
        f.write('tokens,labels\n')
        # for each sequence (sentence) in the test dataset
        for i in range(len(all_toks[:100])):
            toks = all_toks[i]
            if len(toks) == 0:
                continue
            N = len(toks)

            # N x prev_tag x cur_tag
            Y_pred = np.zeros((N, L + 1, L))

            # for each token (word) in the sentence
            for j in range(len(toks)):

                # for each preceding tag of the word
                for possible_previous_tag in label_vocab:
                    X = sparse.lil_matrix((1, num_features))

                    feats = get_features(j, toks, possible_previous_tag, data)
                    valid_feats = {}
                    for feat in feats:
                        if feat in feature_vocab:
                            X[0, feature_vocab[feat]] = feats[feat]

                    # update Y_pred with the probabilities of all current tags
                    # given the current word, previous tag and other data/feature
                    prob = log_reg.predict_proba(X)
                    Y_pred[j, label_vocab[possible_previous_tag]] = prob

            # decode to get the predictions
            predictions = decode(Y_pred)
            for k in range(len(predictions)):
                f.write(str(n) + ',' + index_to_label[predictions[k]] + '\n')
                n += 1
            f.write('\n')
            print("Generation predictions, %.2f%% complete." % ((i+1)/len(all_toks[:100])*100), end="\r")

In [11]:
write_predictions(TEST_FILE, OUTPUT_CSV, log_reg, data)